In [30]:
import pandas as pd
import requests
import json
import folium
from folium import IFrame
from folium.plugins import TimestampedGeoJson, MarkerCluster, LocateControl
import webbrowser

In [31]:
# Request Static Station Data: Caractéristiques et localisation des stations Vélib

r_1 = requests.get("https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/station_information.json")
data_static = r_1.json()

# Request Dynamic Station Data: Le nombre de vélos et de bornettes disponibles par station

r_2 = requests.get("https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/station_status.json")
data_dynamic = r_2.json()

In [32]:
# Extract nested values from a JSON tree formula

def json_extract(obj, key):
    arr = []

    def extract(obj, arr, key):
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key)
    return values

In [33]:
# Extract nested values from a JSON tree: Static Vélib Data 

station_id = json_extract(data_static, 'station_id')
name = json_extract(data_static, 'name')
lat = json_extract(data_static, 'lat')
lon = json_extract(data_static, 'lon')
capacity = json_extract(data_static, 'capacity')
stationCode = json_extract(data_static, 'stationCode')

lists = [station_id, name, lat, lon, capacity, stationCode]
data_velib = pd.concat([pd.Series(x) for x in lists], axis=1)

data_velib = data_velib.rename(columns={0:"station_id", 1:"name", 2:"lat", 3:"lon", 4:"capacity", 5:"stationCode"})


# Extract nested values from a JSON tree: Dynamic Vélib Data

stationCode_d = json_extract(data_dynamic, 'stationCode')
station_id_d = json_extract(data_dynamic, 'station_id')
num_bikes_available = json_extract(data_dynamic, 'num_bikes_available')
numBikesAvailable= json_extract(data_dynamic, 'numBikesAvailable')
mechanical = json_extract(data_dynamic, 'mechanical')
ebike = json_extract(data_dynamic, 'ebike')
num_docks_available = json_extract(data_dynamic, 'num_docks_available')
numDocksAvailable = json_extract(data_dynamic, 'numDocksAvailable')
is_installed = json_extract(data_dynamic, 'is_installed')
is_returning = json_extract(data_dynamic, 'is_returning')
is_renting = json_extract(data_dynamic, 'is_renting')
last_reported = json_extract(data_dynamic, 'last_reported')

lists = [stationCode_d, station_id_d, num_bikes_available, numBikesAvailable, mechanical, ebike, num_docks_available, numDocksAvailable, is_installed, is_returning, is_renting, last_reported]
data_availability = pd.concat([pd.Series(x) for x in lists], axis=1)

data_availability = data_availability.rename(columns={0:"stationCode", 1:"station_id", 2:"num_bikes_available", 3:"numBikesAvailable", 4:"mechanical", 5:"ebike",  6:"num_docks_available",  7:"numDocksAvailable",  8:"is_installed",  9:"is_returning",  10:"is_renting", 11:"last_reported"})

# Merge both dataframes

hidalgo_dataset = pd.merge(data_velib, data_availability, on = "station_id", how = "left")
hidalgo_dataset = hidalgo_dataset.drop(["stationCode_x", "stationCode_y"], axis=1)

print(hidalgo_dataset.head())
print(hidalgo_dataset.describe())

   station_id                                 name        lat       lon  \
0   213688169        Benjamin Godard - Victor Hugo  48.865983  2.275725   
1    99950133   André Mazet - Saint-André des Arts  48.853756  2.339096   
2   516709288  Charonne - Robert et Sonia Delauney  48.855908  2.392571   
3       36255                   Toudouze - Clauzel  48.879296  2.337360   
4    37815204                      Mairie du 12ème  48.840855  2.387555   

   capacity  num_bikes_available  numBikesAvailable  mechanical  ebike  \
0        35                   16                 16          13      3   
1        55                   25                 25          19      6   
2        20                    2                  2           1      1   
3        21                    2                  2           2      0   
4        30                    0                  0           0      0   

   num_docks_available  numDocksAvailable  is_installed  is_returning  \
0                   19         

In [50]:
 # Style marker clusters function

 icon_create_function="""function(cluster) {
		return L.divIcon({ html: '<div style="width:33px;height:33px;text-align:center;border-radius:40px;font-size:13px;font-weight:700;color:#F5F5F5;background-color:#20B2AA">' + cluster.getChildCount() + '</div>', iconSize: new L.Point(1, 1)});
	} """

In [51]:
# Plot & style Paris map 

paris_map = folium.Map(location=[48.856614, 2.3522219],
                      zoom_start=12, tiles="Stamen Toner")

LocateControl().add_to(paris_map)

locations_stations = hidalgo_dataset[["lat", "lon"]]
locations_list = locations_stations.values.tolist()
len(locations_list)

marker_cluster = MarkerCluster(icon_create_function = icon_create_function).add_to(paris_map)


for point in range(0, len(locations_list)):
    folium.Marker(location=locations_list[point], popup=folium.Popup(("<strong>" + "Station: " + "</strong>" + str(data_velib["name"][point]) + "<br>" + "<br>" + "<strong>" + "Mechanical Bikes Available: " + "</strong>" + str(hidalgo_dataset["mechanical"][point]) + "<br>" + "<br>" + "<strong>" + "Ebikes Available: " + "</strong>" + str(hidalgo_dataset["ebike"][point]) + "<br>" + "<br>" + "<strong>" + "Docks Available: " + "</strong>" + str(hidalgo_dataset["num_docks_available"][point])), max_width=180), icon=folium.Icon(color="red",icon="bicycle", icon_color="white", prefix="fa")).add_to(marker_cluster)

paris_map


In [53]:
# Open map in webbrowser 

output_file = "paris_map.html"
paris_map.save(output_file)
webbrowser.open(output_file, new=2)

True